In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

def solve_natural_convection(N, Ra):
    Pr = 0.7  # Prandtl number
    L = 1.0  # Cavity size
    dx = dy = L / (N - 1)
    dt = min(0.25 * dx**2 / Pr, 0.25 * dy**2 / Pr)  # Time step
    max_iter = 5000  # Number of iterations
    tol = 1e-5  # Convergence tolerance
    
    # Initialize fields
    psi = np.zeros((N, N))  # Stream function
    omega = np.zeros((N, N))  # Vorticity
    T = np.zeros((N, N))  # Temperature
    
    # Set initial conditions
    x = np.linspace(0, L, N)
    y = np.linspace(0, L, N)
    X, Y = np.meshgrid(x, y, indexing='ij')
    T[:, 0] = 0.5 * np.cos(np.pi * X[:, 0]) + 1  # Bottom boundary condition
    
    # Time-stepping loop
    for step in range(max_iter):
        T_old = T.copy()
        
        # Update vorticity using finite difference method
        omega[1:-1, 1:-1] = (
            (psi[2:, 1:-1] - 2 * psi[1:-1, 1:-1] + psi[:-2, 1:-1]) / dx**2 +
            (psi[1:-1, 2:] - 2 * psi[1:-1, 1:-1] + psi[1:-1, :-2]) / dy**2
        )
        
        # Solve for stream function using direct solver
        A = diags([-1, 4, -1], [-1, 0, 1], shape=(N-2, N-2)).toarray()
        for j in range(1, N-1):
            psi[1:-1, j] = spsolve(A, omega[1:-1, j])
        
        # Update temperature
        T[1:-1, 1:-1] += dt * (
            - (psi[1:-1, 2:] - psi[1:-1, :-2]) * (T_old[2:, 1:-1] - T_old[:-2, 1:-1]) / (2 * dx * dy) 
            + (T_old[2:, 1:-1] - 2 * T_old[1:-1, 1:-1] + T_old[:-2, 1:-1]) / dx**2 
            + (T_old[1:-1, 2:] - 2 * T_old[1:-1, 1:-1] + T_old[1:-1, :-2]) / dy**2
        )
        
        # Adiabatic conditions on the left and right walls
        T[0, :] = T[1, :]
        T[-1, :] = T[-2, :]
        
        # Convergence check
        if np.linalg.norm(T - T_old, ord=np.inf) < tol:
            print(f'Converged after {step} iterations for Ra = {Ra}')
            break
    
    return X, Y, T

# Solve for both Rayleigh numbers
Ra_values = [3.5e3, 2.5e4]
grid_sizes = [61, 121]

for Ra in Ra_values:
    for N in grid_sizes:
        X, Y, T = solve_natural_convection(N, Ra)
        plt.figure(figsize=(6, 5))
        plt.contourf(X, Y, T, 20, cmap='jet')
        plt.colorbar(label='Temperature')
        plt.title(f'Temperature Contours (Ra={Ra}, Grid={N}x{N})')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.show()


/var/folders/jr/wyy09fn168z4g0q0przyty140000gn/T/ipykernel_97667/3292951246.py:38: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  psi[1:-1, j] = spsolve(A, omega[1:-1, j])


KeyboardInterrupt: 